## Предобработка данных

In [1]:
# Подключение к Google drive

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 16  Cl

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [4]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [5]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [6]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [7]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [8]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [9]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [10]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [11]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [12]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [13]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [14]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [15]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [16]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [17]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [18]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [19]:
#import plotly.express as px
#fig = px.scatter(df, x='ClaimsCount', y='ClaimAmount', title='Зависимость между частотой и величиной убытков')
#fig.show()

Для моделирования среднего убытка можем рассчитать его как отношение величины убытков к их частоте.

In [20]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

## Разделение набора данных на обучающую, валидационную и тестовую выборки

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Разбиение датасета для частоты на train/val/test

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimsCount, test_size=0.3, random_state=1)
x_valid_c, x_test_c, y_valid_c, y_test_c = train_test_split(x_test_c, y_test_c, test_size=0.5, random_state=1)

In [23]:
# Разбиение датасета для среднего убытка на train/val/test 

x_train_ac, x_test_ac, y_train_ac, y_test_ac = train_test_split(dfAC.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'AvgClaim'], axis=1), dfAC.AvgClaim, test_size=0.3, random_state=1)
x_valid_ac, x_test_ac, y_valid_ac, y_test_ac = train_test_split(x_test_ac, y_test_ac, test_size=0.5, random_state=1)

## Обобщенные линейные модели (Generalized Linear Models, GLM)

### Теория

Пусть $y$ – целевая переменная, $X$ – матрица объясняющих переменных, $\beta$ – вектор параметров модели.

Матрица $X$ составлена из всех векторов наблюдений $x_i$, каждый из которых представляет собой объясняющую переменную.

Основные компоненты обобщенной линейной модели:
* Систематическая компонента $\eta$:
    * $\eta = X\beta \hspace{10pt}(=\beta_0+\beta_1x_1+\beta_2x_2+\dots+\beta_nx_n)$.
* Случайная компонента $y$:
    * Элементы вектора $y$ – независимые одинаково распределенные случайные величины, имеющие функцию плотности распределения $f(y;\theta,\phi)$ из экспоненциального семейства.
    * Распределения из экспоненциального семейства имеют параметры $\theta$ (характеристика среднего) и $\phi$ (характеристика дисперсии). В общем виде данные распределения могут быть определены:
    $$f_i(y_i;\theta_i,\phi)=\exp\left\lbrace \frac{y_i\theta_i-b(\theta_i)}{a_i(\phi)} + c(y_i, \phi) \right\rbrace,$$
    где $a_i(\phi)$, $b(\theta_i)$ и $c(y_i, \phi)$ некоторые функции.
    * Для распределений из данного семейства дисперсия является функцией от среднего.
    * Экспоненциальное семейство включает распределения нормальное, экспоненциальное, Пуассона, гамма, хи-квадрат, бета и другие.
* Функция связи $g$:
    * $\mathbb{E}\left[y\right]=\mu=g^{-1}\left(\eta\right)$, $\mu$ – математическое ожидание $y$;
    * $g$ – монотонная дифференцируемая функция.




#### GLM с распределением Пуассона

Регрессия Пуассона обычно используется в случаях, когда зависимая переменная представляет собой счетные значения и ошибки предполагаются распределенными в соответствии с распределением Пуассона. Зависимая переменная должна быть неотрицательной.

Функции связи между таргетом и объясняющими переменными предполагается логарифмической:
$$g(\eta) = \ln(\eta) \Rightarrow \hat {y} = e^{x{^T}\beta + {\beta_{0}}}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} \frac{1}{N} \sum_{i=1}^{N} \Big( y_i(x_{i}^{T}\beta + \beta_0) - e^{x{^T_i}\beta + {\beta_0}} \Big)- \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Тогда соответствующая метрика _Deviance_ имеет вид:
$$D = -2 \sum_{i=1}^{N} \big( y_i \text{ln}(y_i / \hat {y}_i) - (y_i - \hat {y}_i) \big).$$

##### Вывод функции правдоподобия для GLM с распределением Пуассона (без регуляризации)

**NB:** Ниже $\lambda$ не имеет отношения к вышеупомянутому одноименному параметру регуляризации.

Напомним, что функция вероятности для распределения Пуассона имеет вид:
$$p(k;\lambda) = \frac{\lambda^k e^{-\lambda}}{k!}, \hspace{10pt} \lambda\in\mathbb{R}^{+}.$$
Также, для распределения Пуассона справедливо, что:
$$\mathbb{E}\left[k\right] = Var(k) = \lambda.$$
Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют распределение Пуассона:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{e^{y_i(x_i{^T}\beta + {\beta_{0}})} e^{-e^{x_i{^T}\beta + {\beta_{0}}}}}{y_i!} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}-\ln(y_i!)\right).$$
Поскольку величина $\ln(y_i!)$ не зависит от выбора параметров, можно упростить задачу:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}\right).$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с распределением Пуассона

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = \sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i-\ln(y_i!)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = \sum_{i=1}^{N}\left(y_i \ln(\hat{y}_i) -\hat{y}_i-\ln(y_i!)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i - y_i \ln(\hat{y}_i) +\hat{y}_i\right) = -2\sum_{i=1}^{N}\left(y_i \ln(y_i/\hat{y}_i) - (y_i -\hat{y}_i)\right).$$


#### GLM с гамма-распределением

GLM с гамма-распределением используется для моделирования положительной непрерывной зависимой переменной, когда ее условная дисперсия увеличивается вместе со средним значением, но коэффициент вариации зависимой переменной предполагается постоянным.

Обычно GLM с гамма-распределением используются с логарифмической или обратной функциями связи:
$$g(\eta) = \ln(\eta);\hspace{20pt}g(\eta) = \frac{1}{\eta}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия (для обратной функции связи) с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} - \frac{1}{N} \sum_{i=1}^{N} \frac{y_i}{x{^T_i}\beta + \beta_0} + \text{ln} \big( x{^T_i}\beta + \beta_0 \big ) - \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Соответствующая метрика _Deviance_ имеет вид:
$$D = 2 \sum_{i=1}^{N} - \text{ln} \bigg (\dfrac {y_i} {\hat {y}_i} \bigg) + \dfrac {(y_i - \hat{y}_i)} {\hat {y}_i}.$$




##### Вывод функции правдоподобия для GLM с Гамма-распределением и логарифмической функцией связи (без регуляризации)

**NB:** Ниже $\alpha$ и $\lambda$ не имеют отношения к вышеупомянутым одноименным параметрам регуляризации.

Напомним, что функция плотности вероятности для Гамма-распределения имеет вид:
$$f(x;\alpha,\lambda) = 
\begin{cases}
\frac{\alpha^\lambda}{\Gamma(\lambda)}x^{\lambda-1}e^{-\alpha x},&x \ge 0\\
0,& x <0
\end{cases},\hspace{20pt} \Gamma(x) = \int_0^{\infty}x^{\lambda-1}e^{-x}dx.
$$
Также, для Гамма-распределения справедливо, что:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha},\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2}.$$

Для оценивания GLM удобно параметризовать данное распределение иначе:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha} = \mu,\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2} = \frac{\mu^2}{\lambda}.$$
Тогда,
$$f(x;\mu,\lambda) = \frac{1}{\Gamma(\lambda)}\alpha^\lambda x^{\lambda-1}e^{-\alpha x} = \frac{1}{\Gamma(\lambda)}\left(\frac{\lambda}{\mu}\right)^\lambda x^{\lambda-1}e^{-\left(\frac{\lambda}{\mu}\right)x} = \frac{1}{x\cdot\Gamma(\lambda)}\left(\frac{\lambda x}{\mu}\right)^\lambda e^{-\frac{\lambda x}{\mu}},\hspace{15pt} x\ge 0.$$

Также напомним, что для GLM с функцией связи $\ln(x)$ предсказание (оценка математического ожидания) имеет вид $\hat {y} = e^{x{^T}\beta + {\beta_{0}}}$.


Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют Гамма-распределение, в предположении известного параметра $\lambda$:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{1}{y_i\cdot\Gamma(\lambda)}\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)^\lambda e^{-\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
\begin{align}
\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i)+ \lambda\ln\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i) + \lambda\ln\left(\lambda\right)+\lambda\ln\left(y_i\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left((\lambda-1)\ln\left(y_i\right)-\ln(\Gamma(\lambda)) + \lambda\ln\left(\lambda\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right).
\end{align}
Тогда обозначим слагаемые, не зависящие от параметров модели ($\beta_0$, $\beta$), как некоторую константу $C$, получаем:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(-\lambda\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) +\frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) + C(y_i, \lambda)\right),$$
Таким образом, требуется максимизировать
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = -\sum_{i=1}^{N}\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) + \frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right),$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с Гамма-распределением

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = -\sum_{i=1}^{N}\left(1 + \ln(y_i)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = -\sum_{i=1}^{N}\left(\frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(- 1 - \ln(y_i) + \frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right) = 2\sum_{i=1}^{N}\left(-\ln\left(\frac{y_i}{\hat{y}_i}\right)+\frac{y_i-\hat{y}_i}{\hat{y}_i} \right).$$

#### Дополнительная литература по GLM

*   [P. McCullagh, John A. Nelder _"Generalized Linear Models"_](http://www.utstat.toronto.edu/~brunner/oldclass/2201s11/readings/glmbook.pdf)
*   [D. Anderson, et al. _"A practitioner’s guide to generalized linear models"_](https://www.casact.org/pubs/dpp/dpp04/04dpp1.pdf)
*   [E. Ohlsson, B. Johansson _"Non-life insurance pricing with generalized linear models"_](https://www.springer.com/gp/book/9783642107900)
*   [P. De Jong, G. Heller _"Generalized linear models for insurance data"_](https://feb.kuleuven.be/public/u0017833/boek.pdf)




### Установка H2O на Google Colaboratory и инициализация

In [24]:
#!apt-get install default-jre

In [25]:
#!java -version

In [26]:
#!pip install h2o

In [27]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.6+8-b765.1, mixed mode)
  Starting server from C:\Users\fini1\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\fini1\AppData\Local\Temp\tmp5o67hia2
  JVM stdout: C:\Users\fini1\AppData\Local\Temp\tmp5o67hia2\h2o_fini1_started_from_python.out
  JVM stderr: C:\Users\fini1\AppData\Local\Temp\tmp5o67hia2\h2o_fini1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Yekaterinburg
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_fini1_mrl8n4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.492 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


### Построение GLM для частоты страховых случаев

In [28]:
# Преобразование в H2O-Frame

h2o_train_c = h2o.H2OFrame(pd.concat([x_train_c, y_train_c], axis=1))
h2o_valid_c = h2o.H2OFrame(pd.concat([x_valid_c, y_valid_c], axis=1))
h2o_test_c = h2o.H2OFrame(pd.concat([x_test_c, y_test_c], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [29]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "poisson", link = "Log", nfolds=5)
glm_poisson.train(y="ClaimsCount", x = h2o_train_c.names[1:-1], training_frame = h2o_train_c, validation_frame = h2o_valid_c, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.5, lambda = 1.336E-4 )",20,19,3,Key_Frame__upload_bca606585da2410f7aa3ca69fb924f07.hex


In [31]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.5032977,0.0036341609,0.506035,0.50189,0.5034439,0.49796396,0.50715566
1,mean_residual_deviance,1.2440319,0.013592847,1.2520902,1.2463809,1.2476101,1.2203357,1.2537426
2,mse,0.75432396,0.018925728,0.7638568,0.7577031,0.75681365,0.7220708,0.7711755
3,null_deviance,9086.924,200.83118,9100.537,9041.128,9018.627,8863.906,9410.419
4,r2,0.011629272,0.003464079,0.015062428,0.009494439,0.0109385345,0.007388881,0.015262076
5,residual_deviance,8871.022,161.90842,8844.231,8852.633,8814.573,8703.376,9140.299
6,rmse,0.86846286,0.010967691,0.87398905,0.87046146,0.86995035,0.8497475,0.87816596
7,rmsle,0.4102306,0.0019690949,0.41150242,0.41003385,0.41074452,0.40695354,0.41191873


In [32]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.220675,-1.285319
1,LicAge,0.000194,0.030931
2,Gender,0.003870,0.001874
3,MariStat,-0.115084,-0.041152
4,DrivAge,0.004924,0.073696
5,HasKmLimit,-0.467990,-0.145415
6,BonusMalus,0.012914,0.194712
7,OutUseNb,0.084635,0.056781
8,RiskArea,0.015406,0.034085
9,VehUsage_Private,-0.200181,-0.094939


In [33]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-1.28532,-1.28949,-1.28359,-1.28517,-1.28164,-1.28807
LicAge,0.03093,0.04856,0.04506,0.01601,0.00786,0.04149
Gender,0.00187,0.00000,0.00343,0.00290,0.00177,0.00115
MariStat,-0.04115,-0.04453,-0.04217,-0.04393,-0.04234,-0.03416
DrivAge,0.07370,0.14592,0.00619,0.00082,0.09216,0.06374
HasKmLimit,-0.14541,-0.14525,-0.13536,-0.15137,-0.15171,-0.14323
BonusMalus,0.19471,0.19461,0.19239,0.19761,0.19629,0.19019
OutUseNb,0.05678,0.05027,0.06241,0.04884,0.06669,0.05544
RiskArea,0.03408,0.04276,0.04368,0.03337,0.03203,0.01876
VehUsage_Private,-0.09494,-0.08173,-0.10707,-0.10105,-0.08655,-0.09720


In [34]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_poisson.predict(h2o_train_c).as_data_frame()
c_valid_pred = glm_poisson.predict(h2o_valid_c).as_data_frame()
c_test_pred = glm_poisson.predict(h2o_test_c).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [35]:
# Сохранение обученной модели

model_glm_poisson = h2o.save_model(model=glm_poisson, path="/content/drive/My Drive/Colab Notebooks/", force=True)

In [36]:
model_glm_poisson

'C:\\content\\drive\\My Drive\\Colab Notebooks\\GLM_model_python_1588251408301_1'

### Построение GLM для среднего убытка

In [37]:
# Преобразование в H2O-Frame

h2o_train_ac = h2o.H2OFrame(pd.concat([x_train_ac, y_train_ac], axis=1))
h2o_valid_ac = h2o.H2OFrame(pd.concat([x_valid_ac, y_valid_ac], axis=1))
h2o_test_ac = h2o.H2OFrame(pd.concat([x_test_ac, y_test_ac], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [38]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_gamma = H2OGeneralizedLinearEstimator(family = "gamma", link = "Log", nfolds=5)
glm_gamma.train(y="AvgClaim", x = h2o_train_ac.names[1:-1], training_frame = h2o_train_ac, validation_frame = h2o_valid_ac, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [39]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_gamma.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gamma,log,"Elastic Net (alpha = 0.5, lambda = 1.784E-4 )",20,18,3,Key_Frame__upload_8dfd21559d30b089ee7d49e0854f4d50.hex


In [40]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_gamma.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1043.513,42.572292,976.50696,1084.5424,1055.9698,1070.6421,1029.9033
1,mean_residual_deviance,1.9574722,0.06110609,1.8712351,2.0346165,1.9414595,1.9921179,1.9479318
2,mse,7354208.5,7070311.5,2508499.5,6274088.5,4644123.0,1.9755346E7,3588986.5
3,null_deviance,1787.918,76.47664,1732.7142,1887.8292,1694.4679,1797.3865,1827.192
4,r2,-9.474682E-4,0.0038908662,-0.0039145956,-7.598335E-4,-0.0051206443,2.781879E-4,0.004779544
5,residual_deviance,1787.6354,76.43956,1719.5248,1898.3257,1711.6995,1795.4241,1813.2031
6,rmse,2516.5642,1129.7751,1583.8243,2504.813,2155.0227,4444.6987,1894.462
7,rmsle,1.7126696,0.045548283,1.7739097,1.697543,1.6748492,1.670743,1.746303


In [41]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_gamma._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,7.447459,6.923914
1,LicAge,-0.000361,-0.057896
2,Gender,-0.044813,-0.021696
3,MariStat,0.152243,0.056025
4,DrivAge,-0.026294,-0.389649
5,HasKmLimit,-0.086212,-0.021160
6,BonusMalus,-0.003733,-0.062010
7,OutUseNb,0.005180,0.003776
8,RiskArea,0.040690,0.091382
9,VehUsage_Private,0.044360,0.020112


In [42]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_gamma.coef_norm()
for x in range(len(glm_gamma.cross_validation_models())):
    pmodels[x] = glm_gamma.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,6.92391,6.95202,6.90835,6.91050,6.91350,6.92851
LicAge,-0.05790,-0.02513,-0.00775,-0.11555,-0.04535,-0.09699
Gender,-0.02170,-0.02326,-0.02498,-0.04337,0.00211,-0.02000
MariStat,0.05603,0.05271,0.08074,0.06448,0.01229,0.06675
DrivAge,-0.38965,-0.38101,-0.45682,-0.45346,-0.45979,-0.19493
HasKmLimit,-0.02116,-0.02599,-0.02729,-0.01681,-0.01362,-0.02028
BonusMalus,-0.06201,-0.03918,-0.04927,-0.09090,-0.04714,-0.08184
OutUseNb,0.00378,0.00482,0.01278,0.01064,0.00063,-0.00923
RiskArea,0.09138,0.09323,0.07495,0.09466,0.10993,0.08336
VehUsage_Private,0.02011,-0.01025,0.04525,0.03496,0.01155,0.00049


In [43]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ac_train_pred = glm_gamma.predict(h2o_train_ac).as_data_frame()
ac_valid_pred = glm_gamma.predict(h2o_valid_ac).as_data_frame()
ac_test_pred = glm_gamma.predict(h2o_test_ac).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [44]:
# Сохранение обученной модели

model_glm_gamma = h2o.save_model(model=glm_gamma, path="/content/drive/My Drive/Colab Notebooks/", force=True)

In [45]:
model_glm_gamma

'C:\\content\\drive\\My Drive\\Colab Notebooks\\GLM_model_python_1588251408301_2'

### Использование GLM моделей

In [46]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [47]:
df['CountPredicted'] = glm_poisson.predict(h2o_df).as_data_frame()
df['AvgClaimPredicted'] = glm_gamma.predict(h2o_df).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [48]:
df['BurningCost'] = df.CountPredicted * df.AvgClaimPredicted
df[['CountPredicted', 'AvgClaimPredicted', 'BurningCost']].head()

,CountPredicted,AvgClaimPredicted,BurningCost
0,0.365203,1040.884165,380.134137
1,0.365274,1040.508282,380.070571
2,0.321158,1000.561543,321.338065
3,0.264449,959.301700,253.686352
4,0.233909,976.884554,228.501760


## * Домашнее задание: GLM для прогнозирования наступления страхового случая

In [49]:
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,OutUseNb,...,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq,ClaimsCount,CountPredicted,AvgClaimPredicted,BurningCost
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,0,0,1,0,0,2116,0.0,0.365203,1040.884165,380.134137
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,0,0,1,0,0,2116,0.0,0.365274,1040.508282,380.070571
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,1,0,0,1024,0.0,0.321158,1000.561543,321.338065
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,1,0,0,2704,0.0,0.264449,959.301700,253.686352
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,1,0,3249,0.0,0.233909,976.884554,228.501760


In [50]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'CountPredicted', 'AvgClaimPredicted', 'BurningCost'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [51]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [52]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd'].asfactor()
h2o_test_ind['ClaimInd'].asfactor()

ClaimInd
0
0
0
0
0
0
0
0
0
0


In [53]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "poisson", link = "Log", nfolds=5)
glm_poisson.train(y="ClaimInd", x = h2o_train_c.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [54]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.5, lambda = 3.295E-5 )",20,20,3,Key_Frame__upload_aa4e024f254ff5fafe3554de017f4ba2.hex


In [55]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.22225346,0.00124108,0.22225128,0.22182442,0.22279382,0.22388399,0.22051379
1,mean_residual_deviance,0.52145964,0.0045234407,0.52362716,0.51951426,0.52186054,0.52718997,0.5151062
2,mse,0.11118992,0.0015946563,0.11172294,0.110409975,0.1116615,0.11320263,0.10895253
3,null_deviance,3758.0095,35.845463,3787.3796,3742.0906,3782.7017,3775.5874,3702.288
4,r2,0.006083397,0.0012705876,0.004999097,0.0050295433,0.008122199,0.0060637696,0.0062023774
5,residual_deviance,3718.3938,35.63941,3754.8623,3706.9353,3731.7043,3736.0303,3662.4365
6,rmse,0.33344465,0.0023929742,0.33424982,0.33227995,0.3341579,0.33645597,0.3300796
7,rmsle,0.2333488,0.0013762992,0.23380731,0.23276049,0.23367988,0.23510432,0.23139203


In [56]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.271913,-2.077447
1,LicAge,-0.000158,-0.025227
2,Gender,0.013522,0.006548
3,MariStat,-0.046118,-0.016491
4,DrivAge,-0.004404,-0.065912
5,HasKmLimit,-0.382698,-0.118913
6,BonusMalus,0.005558,0.083797
7,OutUseNb,0.058159,0.039018
8,RiskArea,0.012941,0.028631
9,VehUsage_Private,-0.118031,-0.055978


In [57]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-2.07745,-2.07988,-2.07588,-2.07821,-2.08304,-2.07151
LicAge,-0.02523,-0.03939,-0.00717,-0.04417,-0.02715,-0.00095
Gender,0.00655,0.01166,0.00598,0.00518,-0.00353,0.01384
MariStat,-0.01649,-0.01473,-0.02342,-0.00984,-0.01862,-0.01696
DrivAge,-0.06591,-0.04603,-0.13431,-0.01654,-0.05310,-0.13765
HasKmLimit,-0.11891,-0.12900,-0.11691,-0.11528,-0.11399,-0.11982
BonusMalus,0.08380,0.08218,0.08877,0.07977,0.08560,0.08016
OutUseNb,0.03902,0.04442,0.04236,0.03654,0.03148,0.04022
RiskArea,0.02863,0.02538,0.03434,0.02386,0.02263,0.03711
VehUsage_Private,-0.05598,-0.04179,-0.05722,-0.05619,-0.07132,-0.04395


In [58]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_poisson.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_poisson.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_poisson.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [59]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score

In [60]:
y_train_ind[y_train_ind==1].sum() / y_train_ind.shape[0]

0.09471764589122668

In [61]:
# confusion_matrix(y_true, y_pred)

#что-то пошло не так

In [62]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

for label, y_true, y_pred in [('Train Dataset', y_train_ind, ind_train_pred),
                              ('Valid Dataset', y_valid_ind, ind_valid_pred),
                              ('Test Dataset', y_test_ind, ind_test_pred)]:
    print(label.upper())
    print('class 1%:', y_train_ind[y_train_ind==1].sum() / y_train_ind.shape[0]*100, 1)
    print('Accuracy: ', round(accuracy_score(y_true, round(y_pred)),4))
    print('f1: ', round(f1_score(y_true, round(y_pred)),4))
    print('precision: ', round(precision_score(y_true, round(y_pred)),4))
    print('recall: ', round(recall_score(y_true, round(y_pred)),4))
    print('ROC AUC: ', round(roc_auc_score(y_true, y_pred),4), '\n')

TRAIN DATASET
class 1%: 9.471764589122667 1
Accuracy:  0.9053
f1:  0.0
precision:  0.0
recall:  0.0
ROC AUC:  0.5653 

VALID DATASET
class 1%: 9.471764589122667 1
Accuracy:  0.9068
f1:  0.0
precision:  0.0
recall:  0.0
ROC AUC:  0.564 

TEST DATASET
class 1%: 9.471764589122667 1
Accuracy:  0.9059
f1:  0.0
precision:  0.0
recall:  0.0
ROC AUC:  0.5685 



Какие проблемы вы здесь видите? Как можно улучшить данный результат?

Итак, учитывая что ROC AUC имеет значение около 0.5 к которому стремятся случайные значения (и это при условии, что обычно ROC AUC устойчива к несбалансированным данным), а метрики f1, precision, recall равны 0.0, можно одназначно утверждать, что наша модель, из-за сильного дисбаланса классов, показала низкое качество результатов для данного датасета. Улучшить полученный результат возможно за счет увеличения количества данных для обучения нашей модели или требует значительной доработки модели.